# Make Python Sing!

In [1]:
import numpy as np
from IPython.display import Audio

In [2]:
sampling_Hz = 44100

In [3]:
def note_to_frequency(note):
    """
    Specify the note as a letter note, optional modifier (b,bb,#,##), and octave number.
    """
    note_letter = note[0].upper()
    octave = int(note[-1])
    half_step_mod = 0
    if len(note) > 2:
        modifier = note[1:-1]
        half_step_mod = {
            "b":-1,
            "bb":-2,
            "#":1,
            "##":2
        }[modifier]

    # reference frequency is A4 = 440 Hz
    a4 = 440
    # calculate which octave we are in relative to a4
    octaves_from_a4 = octave - 4
    # calculate how many half steps this note letter is from A in the
    # same octave
    note_letter_distance = {
        "A": 0,
        "B": 2,
        "C": -9,
        "D": -7,
        "E": -5,
        "F": -4,
        "G": -2,
    }[note_letter]
    # calculate how many half steps away we are total
    total_half_steps = octaves_from_a4*12 + note_letter_distance + half_step_mod
    # calculate the frequency. For simplicity, just use equal-tempered tuning.
    frequency = a4*(2**(total_half_steps/12))
    return frequency
    
def synthesize_tone(note,length,sampling_Hz=44100):
    """
    Synthesize a single note

    note: string representation of note
    length: length of note in seconds
    sampling_Hz: sampling rate in Hz
    """
    # Get the frequency of this note
    frequency_Hz = note_to_frequency(note)

    # Calculate the time samples in seconds
    t = np.arange(0,length,1.0/sampling_Hz)

    # Convert to radians per second
    frequency_rad_per_second = 2*np.pi*frequency_Hz

    # Calculate the pure tone, which is a simple sinusoid
    tone = np.cos(frequency_rad_per_second*t)

    # Multiply the begining and end by a ramp so we don't
    # have a harsh onset and offset
    onset_time=0.05
    ramp = np.linspace(0,1,int(onset_time*sampling_Hz))
    tone[0:len(ramp)] = tone[0:len(ramp)]*ramp
    tone[-len(ramp):] = tone[-len(ramp):]*ramp[::-1]

    return tone

def synthesize_melody(note_tuples,sampling_Hz=44100,seconds_per_beat=0.5):
    """
    Synthesize a bunch of sequency notes. Specify as an array of touples
    with the note and note length.
    """
    melody = []
    for note_tuple in note_tuples:
        note = note_tuple[0]
        length = note_tuple[1]*seconds_per_beat
        tone = synthesize_tone(note,length,sampling_Hz)
        melody.extend(tone)
    return melody

In [4]:
mary_had_a_little_lamb = [
    ("E4",1),("D4",1),("C4",1),("D4",1),("E4",1),("E4",1),("E4",2),
    ("D4",1),("D4",1),("D4",2),("E4",1),("G4",1),("G4",2),
    ("E4",1),("D4",1),("C4",1),("D4",1),("E4",1),("E4",1),("E4",1),
    ("E4",1),("D4",1),("D4",1),("E4",1),("D4",1),("C4",2)
]
melody = synthesize_melody(mary_had_a_little_lamb,sampling_Hz)
Audio(data=melody,rate=sampling_Hz,autoplay=False)